In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, concatenate, BatchNormalization, MaxPooling2D, UpSampling2D, Concatenate, Dropout, Cropping2D, ZeroPadding2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import jaccard_score
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.metrics import MeanIoU
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from transformers import ViTModel, ViTConfig
import torchvision.models as models

In [2]:
TRAIN_IMAGE_DIR = "/kaggle/input/diabetic-retinopathy-dataset/Daataset_DR/DB0/Images"
TRAIN_MASK_DIR = "/kaggle/input/diabetic-retinopathy-dataset/Daataset_DR/DB0/GroundTruth"
TEST_IMAGE_DIR = "/kaggle/input/diabetic-retinopathy-dataset/Daataset_DR/DB1/Images"
TEST_MASK_DIR = "/kaggle/input/diabetic-retinopathy-dataset/Daataset_DR/DB1/GroundTruth"

In [3]:
def create_combined_mask(image_filename):
    sample_mask_path = os.path.join(TRAIN_MASK_DIR, mask_subfolders[0], image_filename)
    mask_shape = cv2.imread(sample_mask_path, cv2.IMREAD_GRAYSCALE).shape
    combined_mask = np.zeros(mask_shape, dtype=np.uint8)

    for subfolder in mask_subfolders:
        mask_path = os.path.join(TRAIN_MASK_DIR, subfolder, image_filename)
        if os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            combined_mask = cv2.bitwise_or(combined_mask, mask)

    return combined_mask

In [4]:
OUTPUT_MASK_DIR = "/kaggle/working/CombinedMasksTask1"
os.makedirs(OUTPUT_MASK_DIR, exist_ok=True)
mask_subfolders = ['CWS', 'HE']
image_filenames = os.listdir(TRAIN_IMAGE_DIR)

for image_filename in tqdm(image_filenames):
    combined_mask = create_combined_mask(image_filename)
    output_path = os.path.join(OUTPUT_MASK_DIR, image_filename)
    cv2.imwrite(output_path, combined_mask)

100%|██████████| 130/130 [00:05<00:00, 23.98it/s]


In [5]:
OUTPUT_MASK_DIR = "/kaggle/working/CombinedMasksTask2"
os.makedirs(OUTPUT_MASK_DIR, exist_ok=True)
mask_subfolders = ['H', 'MA']
image_filenames = sorted(os.listdir(TRAIN_IMAGE_DIR))

for image_filename in tqdm(image_filenames):
    combined_mask = create_combined_mask(image_filename)
    output_path = os.path.join(OUTPUT_MASK_DIR, image_filename)
    cv2.imwrite(output_path, combined_mask)

100%|██████████| 130/130 [00:05<00:00, 25.15it/s]


In [6]:
OUTPUT_MASK_DIR = "/kaggle/working/TestCombinedMasksTask1"
os.makedirs(OUTPUT_MASK_DIR, exist_ok=True)
mask_subfolders = ['CWS', 'HE']
image_filenames = os.listdir(TEST_IMAGE_DIR)

for image_filename in tqdm(image_filenames):
    combined_mask = create_combined_mask(image_filename)
    output_path = os.path.join(OUTPUT_MASK_DIR, image_filename)
    cv2.imwrite(output_path, combined_mask)

100%|██████████| 89/89 [00:02<00:00, 42.55it/s]


In [7]:
OUTPUT_MASK_DIR = "/kaggle/working/TestCombinedMasksTask2"
os.makedirs(OUTPUT_MASK_DIR, exist_ok=True)
mask_subfolders = ['H', 'MA']
image_filenames = os.listdir(TEST_IMAGE_DIR)

for image_filename in tqdm(image_filenames):
    combined_mask = create_combined_mask(image_filename)
    output_path = os.path.join(OUTPUT_MASK_DIR, image_filename)
    cv2.imwrite(output_path, combined_mask)

100%|██████████| 89/89 [00:02<00:00, 42.49it/s]


In [10]:
IMAGE_DIR = "/kaggle/input/diabetic-retinopathy-dataset/Daataset_DR/DB0/Images"
MASK_DIR = "/kaggle/working/CombinedMasksTask1"

IMG_HEIGHT = 1152
IMG_WIDTH = 1500
BATCH_SIZE = 1
EPOCHS = 50

def load_data(image_dir, mask_dir):
    image_filenames = sorted(os.listdir(image_dir))
    images = []
    masks = []
    
    for filename in tqdm(image_filenames, desc="Loading data"):
        # Load and resize image
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        #image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        images.append(image)  # Normalize image to [0, 1]
        
        # Load and resize mask
        mask_path = os.path.join(mask_dir, filename)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        #mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
        mask = (mask > 0).astype(np.uint8)  # Binarize mask (0 or 1)
        masks.append(mask)
    
    return np.array(images), np.array(masks).reshape(-1, IMG_HEIGHT, IMG_WIDTH, 1)

images, masks = load_data(IMAGE_DIR, MASK_DIR)

X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.1, random_state=42)

Loading data: 100%|██████████| 130/130 [00:07<00:00, 17.60it/s]


# ***UNET***

In [11]:
def build_unet(input_shape=(1152, 1500, 3)):
    inputs = Input(shape=input_shape)

    x = ZeroPadding2D(padding=((0,0),(2,2)))(inputs)  # shape => (1152, 1504, 3)

    # -------- Encoder --------
    c1 = Conv2D(64, 3, activation='relu', padding='same')(x)
    c1 = Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D(pool_size=(2, 2))(c1)  # shape => (576, 752)

    c2 = Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D(pool_size=(2, 2))(c2)  # (288, 376)

    c3 = Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = Conv2D(256, 3, activation='relu', padding='same')(c3)
    p3 = MaxPooling2D(pool_size=(2, 2))(c3)  # (144, 188)

    c4 = Conv2D(512, 3, activation='relu', padding='same')(p3)
    c4 = Conv2D(512, 3, activation='relu', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)  # (72, 94)

    # -------- Bottleneck --------
    c5 = Conv2D(1024, 3, activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, 3, activation='relu', padding='same')(c5)

    # -------- Decoder --------
    u6 = UpSampling2D((2, 2))(c5)           # (144, 188)
    u6 = concatenate([u6, c4])             # both (144,188) => OK
    c6 = Conv2D(512, 3, activation='relu', padding='same')(u6)
    c6 = Conv2D(512, 3, activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)          # (288, 376)
    u7 = concatenate([u7, c3])             # both (288,376)
    c7 = Conv2D(256, 3, activation='relu', padding='same')(u7)
    c7 = Conv2D(256, 3, activation='relu', padding='same')(c7)

    u8 = UpSampling2D((2, 2))(c7)          # (576, 752)
    u8 = concatenate([u8, c2])             # both (576,752)
    c8 = Conv2D(128, 3, activation='relu', padding='same')(u8)
    c8 = Conv2D(128, 3, activation='relu', padding='same')(c8)

    u9 = UpSampling2D((2, 2))(c8)          # (1152,1504)
    u9 = concatenate([u9, c1])             # both (1152,1504)
    c9 = Conv2D(64, 3, activation='relu', padding='same')(u9)
    c9 = Conv2D(64, 3, activation='relu', padding='same')(c9)

    c9_cropped = Cropping2D(cropping=((0,0),(2,2)))(c9)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9_cropped)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=[MeanIoU(num_classes=2)]
    )

    return model

In [ ]:
unet_model = build_unet((IMG_HEIGHT, IMG_WIDTH, 3))

checkpoint = ModelCheckpoint("unet_model.keras", save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = unet_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/50


In [ ]:
TEST_IMAGES = "/kaggle/input/diabetic-retinopathy-dataset/Daataset DR/DB1/Images"
TEST_MASKS = "/kaggle/working/TestCombinedMasksTask1"
X_test, y_test = load_data(TEST_IMAGES, TEST_MASKS)
predictions = unet_model.predict(X_test, batch_size=1)
print("Predictions shape:", predictions.shape)  

pred_masks = (predictions > 0.5).astype(np.uint8)
mean_iou = MeanIoU(num_classes=2)
mean_iou.update_state(y_test, pred_masks)
print("Mean IoU on test set:", mean_iou.result().numpy())

In [ ]:
TEST_IMAGE_PATH = "/kaggle/input/diabetic-retinopathy-dataset/Daataset DR/DB1/Images/image089.png"
MODEL_PATH = "/kaggle/working/unet_model.keras"
GROUND_TRUTH_PATH = "/kaggle/working/TestCombinedMasksTask1/image089.png"

# Image dimensions
IMG_HEIGHT = 1152
IMG_WIDTH = 1500

# Load the trained model
model = load_model(MODEL_PATH)

# Load and preprocess the test image
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    original_size = image.shape[:2]  # Save original size for resizing prediction
    image_resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    return image, image_resized.reshape(1, IMG_HEIGHT, IMG_WIDTH, 3), original_size

original_image, preprocessed_image, original_size = load_and_preprocess_image(TEST_IMAGE_PATH)

# Predict the mask
predicted_mask = model.predict(preprocessed_image)[0]
predicted_mask = (predicted_mask > 0.5).astype(np.uint8)  # Threshold to get binary mask

# Resize prediction to original size
predicted_mask_resized = cv2.resize(predicted_mask, (original_size[1], original_size[0]))

# Load ground truth image
ground_truth_image = cv2.imread(GROUND_TRUTH_PATH, cv2.IMREAD_GRAYSCALE)
ground_truth_image_resized = cv2.resize(ground_truth_image, (original_size[1], original_size[0]))

# Plot the results
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Predicted Mask")
plt.imshow(predicted_mask_resized, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Ground Truth Mask")
plt.imshow(ground_truth_image_resized, cmap='gray')
plt.axis('off')

plt.show()